# Análise Sentimental do Review de Jogos

## Projeto da disciplina **SSC0287 - Mineração de Dados Não Estruturados**

- Lucas Ivars Cadima Ciziks - luciziks@usp.br - 12559472

- Gustavo Silva de Oliveira - guspfc03@usp.br - 12567231

---

# 1. Introdução

O objetivo principal desse trabalho é **sumarizar informações textuais** no contexto de **Review de Jogos por Usuários**, utilizando as técnicas de Mineração de Dados Textuais aprendidas no decorrer da disciplina. Dentro de plataformas de games, como a Steam e Epic Games, é comum cada jogo ter uma seção contendo a avaliação geral dos usuários que já jogaram. Desse modo, novos compradores podem ter uma ideia do que esperar de um jogo. 


Nesse contexto, nossa abordagem visa extrair o sentimento geral na opinião de um usuário ao descrever sua experiência jogando determinado jogo, verificando, assim, se a avaliação foi **Positiva**, **Negativa** ou **Neutra**. Para isso, treinamos um modelo base do [BERT](https://huggingface.co/bert-base-uncased) com uma base de avaliações de jogos advindas do Twitter. Desse modo, o modelo é capaz de analisar a linguagem complexa presente nos reviews e traduzi-lá em informação resumida.

Além disso, treinamos um segundo modelo dedicado à detecção de discurso de ódio, classificando o texto como **Discurso de Ódio**, **Linguagem ofensiva** ou **Nada Detectado** nos reviews. Assim, em um caso real como na plataforma Steam, o uso combinado desses modelos permite coletar informações diretas sobre os jogos, além de evitar a disseminação de conteúdos prejudiciais nesses fóruns.

Com perspectiva para futuras melhorias, propomos a implementação da explicabilidade dos modelos. Esse aprimoramento permitirá uma compreensão mais clara sobre os motivos específicos que contribuem para o sentimento geral ser positivo, negativo ou neutro. Ademais, identificar quais palavras específicas foram a razão para se qualificar a review como discurso de ódio ou linguagem ofensiva seria útil para eventuais censuras ou aviso ao usuário.

O projeto foi desenvolvido na linguagem **Python**, utilizando o [Poetry](https://python-poetry.org) para gerenciamento das dependências. Dentre as bibliotecas utilizadas, estão:

* [Pandas](https://pandas.pydata.org);
* [Scikit-learn](https://scikit-learn.org/stable/);
* [Keras](https://keras.io);
* [HuggingFace](https://huggingface.co/docs/huggingface_hub/index).

Antes de rodar o notebook, certifique-se de instalar todas as depências através do comando:

```poetry install```

---

# 2. Pré-Processamento

As bases de dados utilizadas neste projeto podem ser encontradas em:

* [Análise de Sentimentos](https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis);
* [Detecção de Discurso de Ódio](https://www.kaggle.com/datasets/mexwell/hate-speech-identification).

In [ ]:
# Importando Dependências do Projeto

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical

from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Dropout
from keras.initializers import Constant

from transformers import (
    TFBertModel,
    BertTokenizer,
    TFAutoModelForSequenceClassification,
    DataCollatorWithPadding,
)

## 2.1 Análise de Sentimentos

In [ ]:
# lendo a base de dados

sentimental_analysis_data = pd.read_csv("https://raw.githubusercontent.com/ciziks/sentiment-game-reviews/main/training_data/sentimental_analysis_data.csv")

In [ ]:
# nomeando as colunas, removendo valores nulos e tomando apenas as variáveis de interesse

sentimental_analysis_data.columns = ["id", "palavra-chave", "sentimento", "tweet"]

sentimental_analysis_data.dropna(inplace=True)

sentimental_analysis_data = sentimental_analysis_data[["tweet", "sentimento"]]

In [ ]:
# mapeando a variável de interesse para valores numéricos interpretáveis pelo modelo de treinamento (foi tomada a decisão de
# agrupar as respostas "Neutral" e "Irrelevant" na mesma categoria)

mapeamento = {"Positive": 0, "Neutral": 1, "Negative": 2, "Irrelevant": 1}

sentimental_analysis_data["sentimento"] = sentimental_analysis_data["sentimento"].map(mapeamento)

sentimental_analysis_data["sentimento"] = sentimental_analysis_data["sentimento"].astype(int)

# dividindo em datasets de treino e teste

sentimental_analysis_train, sentimental_analysis_test = train_test_split(sentimental_analysis_data, test_size=0.2, random_state = 42)

sentimental_analysis_train.head(10)

In [ ]:
# convertendo os dados para o formato 'Dataset', amplamente utilizado no treinamento de modelos de aprendizdo

sentimental_analysis_train = Dataset.from_pandas(sentimental_analysis_train)
sentimental_analysis_test = Dataset.from_pandas(sentimental_analysis_test)

sentimental_analysis_dataset = DatasetDict({"train": sentimental_analysis_train, "test": sentimental_analysis_test})

## 2.2 Identificação de Discurso de Ódio 

In [ ]:
# lendo a base de dados

hate_speech_data = pd.read_csv("https://raw.githubusercontent.com/ciziks/sentiment-game-reviews/main/training_data/hate_speech_data.csv")

In [ ]:
# tomando apenas as variáveis de interesse e renomeando-as

hate_speech_data = hate_speech_data[["tweet", "class"]]

hate_speech_data.rename(columns={"class": "label", "tweet": "text"}, inplace=True)

In [ ]:
# dividindo em datasets de treino e teste

hate_speech_train, hate_speech_test = train_test_split(hate_speech_data, test_size=0.2, random_state=42)

In [ ]:
# convertendo os dados para o formato 'DatasetDict', amplamente utilizado no treinamento de modelos de aprendizdo

hate_speech_train = Dataset.from_pandas(hate_speech_train)
hate_speech_test = Dataset.from_pandas(hate_speech_test)

hate_speech_dataset = DatasetDict({"train": hate_speech_train, "test": hate_speech_test})

---

# 3. Extração de Padrões

In [ ]:
# carregando o tokenzier do modelo base do BERT

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
# funcao utilizada para tokenizar as entradas

def tokenize_function_sentimental(example):
    return tokenizer(example["tweet"], truncation=True)

def tokenize_function_hate_speech(example):
    return tokenizer(example["text"], truncation=True)

In [ ]:
# carregando o modelo base do BERT

model_sentimental = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=3
)

model_hate_speech = TFAutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=3
)

In [ ]:
# especificando o data collator para TensorFlow

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

## 3.1 Análise de Sentimentos

In [ ]:
# tokenizando o dataset

sentimental_analysis_tokenized = sentimental_analysis_dataset.map(tokenize_function_sentimental, batched=True)

In [ ]:
# convertendo os dados para o formato TensorFlow
# aqui ja se especifica o tamanho do batch em 8

sentimental_analysis_train_tf = sentimental_analysis_tokenized["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["sentimento"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

sentimental_analysis_validation_tf = sentimental_analysis_tokenized["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["sentimento"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
# especificando parâmetros importantes (learning rate, loss, métrica e número de épocas) e treinando o modelo

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]

model_sentimental.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model_sentimental.fit(sentimental_analysis_train_tf, validation_data=sentimental_analysis_validation_tf, epochs=5)

## 3.2 Identificação de Discurso de Ódio

In [ ]:
# tokenizando o dataset

hate_speech_tokenized = hate_speech_dataset.map(tokenize_function_hate_speech, batched=True)

In [ ]:
# convertendo os dados para o formato TensorFlow
# aqui ja se especifica o tamanho do batch em 8

hate_speech_train_tf = hate_speech_tokenized["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

hate_speech_validation_tf = hate_speech_tokenized["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

In [ ]:
# especificando parâmetros importantes (learning rate, loss, métrica e número de épocas) e treinando o modelo

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]

model_hate_speech.compile(optimizer=optimizer, loss=loss, metrics=metrics)

model_hate_speech.fit(hate_speech_train_tf, validation_data=hate_speech_validation_tf, epochs=5)

Neste notebook, por motivos de otimização de tempo, optou-se por não executar os modelos de treinamento. Todavia, é possível encontrar os Notebooks com todas as etapas executadas no [Github](https://github.com/ciziks/sentiment-game-reviews) do projeto.

---

# 4. Pós-Processamento

Após o treinamento, os modelos foram carregados na plataforma *HuggingFace*, que nos permite carregar os modelos já treinados e utilizá-los, sem a necessidade de treinar novamente. Você pode encontrar os modelos em:

* [Modelo para Análises de Sentimentos](https://huggingface.co/Guspfc/my-awesome-bert-model-sentiment-analysis);
* [Modelo para Identificação de Discurso de Ódio](https://huggingface.co/Guspfc/my-awesome-bert-model-sentiment-analysis).

---

# 5. Uso do Conhecimento

A fim utilizar os modelos de **Análise de Sentimento** e **Detecção de Discurso de Ódio** treinados nas etapas anteriores, construímos um site para coletar a opinião do usuário acerca dos nomeados a Game of The Year 2023 e verificar o sentimento geral e se houve discurso de ódio na descrição.

Para isso, utilizamos o [Streamlit](https://docs.streamlit.io), uma biblioteca do Python que facilita a implementação de uma interface gráfica para projetos de Machine Learning e Data Science. O código está disponível no arquivo `app.py` e é possível executá-lo localmente através do comando:

```poetry run streamlit run app.py```
